In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# Load Data

In [2]:
dfFly = pd.read_csv('flights-selected/1912_bts_flights.csv', dtype={'DepTimeBlk':'str','ArrTimeBlk':'str'})

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dfFly.describe()

,DayofMonth,DayOfWeek,Flight_Number_Marketing_Airline,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,TaxiOut,...,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings
count,570016.000000,570016.000000,570016.000000,570016.000000,564376.000000,564376.000000,564376.000000,564376.000000,564376.000000,564198.000000,...,562858.000000,570016.0,570016.000000,570016.000000,113365.000000,113365.000000,113365.000000,113365.000000,113365.000000,570016.000000
mean,15.775298,3.931521,3101.811811,1326.288866,1335.003971,12.223567,14.985285,0.202937,0.151720,18.102063,...,104.643825,1.0,735.643542,3.420746,20.815084,3.381767,14.684276,0.070930,29.052768,0.003724
std,8.970281,2.087392,1801.560199,488.491452,500.886347,51.731247,50.814703,0.402187,2.282292,10.593211,...,65.681620,0.0,550.087864,2.164162,69.213349,34.703872,35.416056,2.152037,57.754517,0.125676
min,1.000000,1.000000,1.000000,1.000000,1.000000,-49.000000,0.000000,0.000000,-2.000000,1.000000,...,8.000000,1.0,45.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,2.000000,1574.000000,915.000000,921.000000,-5.000000,0.000000,0.000000,-1.000000,12.000000,...,58.000000,1.0,337.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,16.000000,4.000000,2943.000000,1320.000000,1328.000000,-1.000000,0.000000,0.000000,-1.000000,15.000000,...,87.000000,1.0,590.000000,3.000000,1.000000,0.000000,1.000000,0.000000,7.000000,0.000000
75%,23.000000,6.000000,4753.000000,1730.000000,1741.000000,9.000000,9.000000,0.000000,0.000000,21.000000,...,133.000000,1.0,966.000000,4.000000,17.000000,0.000000,18.000000,0.000000,35.000000,0.000000
max,31.000000,7.000000,9343.000000,2359.000000,2400.000000,2710.000000,2710.000000,1.000000,12.000000,182.000000,...,701.000000,1.0,4983.000000,11.000000,2695.000000,1847.000000,1237.000000,366.000000,1627.000000,9.000000


#### Looks like the flights are all within one month, so we won't need FlightDate; DayofMonth and DayOfWeek should do

#### We also shouldn't use the delay information we already know to train our model to predict delays. Practically, we wouldn't know that information beforehand

## Set target and features

In [4]:
y = dfFly['ArrDel15']
X = dfFly.drop(['DepDel15','FlightDate','DepDelay','DepDelayMinutes','DepartureDelayGroups','ArrDelay','ArrDelayMinutes','ArrivalDelayGroups',
                'ArrDel15','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay'], axis=1)

# Feature Engineering

In [5]:
X.describe()

,DayofMonth,DayOfWeek,Flight_Number_Marketing_Airline,CRSDepTime,DepTime,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,DivAirportLandings
count,570016.000000,570016.000000,570016.000000,570016.000000,564376.000000,564198.000000,564198.000000,563893.000000,563893.000000,570016.000000,563893.000000,570016.000000,570016.000000,570015.000000,562858.000000,562858.000000,570016.0,570016.000000,570016.000000,570016.000000
mean,15.775298,3.931521,3101.811811,1326.288866,1335.003971,18.102063,1360.436031,1461.241227,7.747665,1484.496047,1465.743382,0.010372,0.002186,136.566915,130.481439,104.643825,1.0,735.643542,3.420746,0.003724
std,8.970281,2.087392,1801.560199,488.491452,500.886347,10.593211,501.339745,528.988057,6.181136,515.002524,533.331895,0.101312,0.046703,67.127130,67.347725,65.681620,0.0,550.087864,2.164162,0.125676
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,-143.000000,25.000000,8.000000,1.0,45.000000,1.000000,0.000000
25%,8.000000,2.000000,1574.000000,915.000000,921.000000,12.000000,937.000000,1049.000000,4.000000,1104.000000,1053.000000,0.000000,0.000000,89.000000,82.000000,58.000000,1.0,337.000000,2.000000,0.000000
50%,16.000000,4.000000,2943.000000,1320.000000,1328.000000,15.000000,1342.000000,1502.000000,6.000000,1514.000000,1505.000000,0.000000,0.000000,119.000000,113.000000,87.000000,1.0,590.000000,3.000000,0.000000
75%,23.000000,6.000000,4753.000000,1730.000000,1741.000000,21.000000,1756.000000,1906.000000,9.000000,1915.000000,1912.000000,0.000000,0.000000,165.000000,159.000000,133.000000,1.0,966.000000,4.000000,0.000000
max,31.000000,7.000000,9343.000000,2359.000000,2400.000000,182.000000,2400.000000,2400.000000,189.000000,2359.000000,2400.000000,1.000000,1.000000,1509.000000,734.000000,701.000000,1.0,4983.000000,11.000000,9.000000


### Have to deal with missing values in target,features in order to establish baseline

In [6]:
y_repnull0 = y.fillna(2.0)

In [7]:
X.dtypes

DayofMonth                                   int64
DayOfWeek                                    int64
Marketing_Airline_Network                   object
Operated_or_Branded_Code_Share_Partners     object
Flight_Number_Marketing_Airline              int64
Origin                                      object
OriginCityName                              object
OriginState                                 object
Dest                                        object
DestCityName                                object
DestState                                   object
CRSDepTime                                   int64
DepTime                                    float64
DepTimeBlk                                  object
TaxiOut                                    float64
WheelsOff                                  float64
WheelsOn                                   float64
TaxiIn                                     float64
CRSArrTime                                   int64
ArrTime                        

In [8]:
# Get all numerical columns
colsNum = [col for col in X.columns if X[col].dtype in ['int64','float64']]
X_num = X.loc[:,colsNum]
X_num

,DayofMonth,DayOfWeek,Flight_Number_Marketing_Airline,CRSDepTime,DepTime,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,Cancelled,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,DivAirportLandings
0,31,2,1583,1820,1823.0,20.0,1843.0,2233.0,6.0,2244,2239.0,0.0,0.0,204.0,196.0,170.0,1.0,1440.0,6,0
1,1,7,1584,1235,1314.0,11.0,1325.0,1401.0,10.0,1344,1411.0,0.0,0.0,69.0,57.0,36.0,1.0,190.0,1,0
2,4,3,1584,1240,1234.0,14.0,1248.0,1326.0,10.0,1348,1336.0,0.0,0.0,68.0,62.0,38.0,1.0,190.0,1,0
3,5,4,1584,1240,1236.0,14.0,1250.0,1333.0,13.0,1348,1346.0,0.0,0.0,68.0,70.0,43.0,1.0,190.0,1,0
4,6,5,1584,1240,1531.0,13.0,1544.0,1617.0,7.0,1348,1624.0,0.0,0.0,68.0,53.0,33.0,1.0,190.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570011,22,7,18,1215,1222.0,8.0,1230.0,1345.0,9.0,1345,1354.0,0.0,0.0,90.0,92.0,75.0,1.0,461.0,2,0
570012,22,7,716,1650,1739.0,9.0,1748.0,1904.0,3.0,1820,1907.0,0.0,0.0,90.0,88.0,76.0,1.0,461.0,2,0
570013,22,7,1525,2200,2201.0,20.0,2221.0,2336.0,4.0,2330,2340.0,0.0,0.0,90.0,99.0,75.0,1.0,461.0,2,0
570014,22,7,4740,605,603.0,11.0,614.0,729.0,5.0,735,734.0,0.0,0.0,90.0,91.0,75.0,1.0,461.0,2,0


#### Need to get rid of the rest of the columns of data we wouldn't know before the flight takes off, so we can make a realistic prediction

In [9]:
X_num_better = X_num.drop(['DepTime','TaxiOut','WheelsOff','WheelsOn','TaxiIn','ArrTime','ActualElapsedTime','AirTime','DivAirportLandings'], axis=1)

### Find useful categorical factors

In [10]:
# Get all categorical columns
colsCat = [col for col in X.columns if X[col].dtype in ['object']]
X_cat = X.loc[:,colsCat]
X_cat.describe()

,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,Origin,OriginCityName,OriginState,Dest,DestCityName,DestState,DepTimeBlk,ArrTimeBlk,CancellationCode
count,570016,570016,570016,570016,570016,570016,570016,570016,570016,570016,5912
unique,4,7,319,315,53,319,315,52,19,19,3
top,AA,WN,ORD,"Chicago, IL",TX,ORD,"Chicago, IL",TX,0600-0659,1100-1159,B
freq,177350,114442,31853,38712,64952,31791,38650,64939,42256,36579,3483


In [11]:
# Drop cols with too many unique values
X_cat_better = X_cat.drop(['Origin','OriginCityName','OriginState','Dest','DestCityName','DestState'], axis=1)

In [12]:
# Combine good categorical and numerical columns into one df
colsGood = [col for col in X_num_better.columns]
for col in X_cat_better.columns:   
    colsGood.append(col)

X_Good = X.loc[:,colsGood]
X_Good.isnull().any()

DayofMonth                                 False
DayOfWeek                                  False
Flight_Number_Marketing_Airline            False
CRSDepTime                                 False
CRSArrTime                                 False
Cancelled                                  False
Diverted                                   False
CRSElapsedTime                              True
Flights                                    False
Distance                                   False
DistanceGroup                              False
Marketing_Airline_Network                  False
Operated_or_Branded_Code_Share_Partners    False
DepTimeBlk                                 False
ArrTimeBlk                                 False
CancellationCode                            True
dtype: bool

In [13]:
# Split X,y into train and test
X_train, X_test, y_train, y_test = train_test_split(X_Good, y_repnull0, test_size=0.2, random_state=1)

In [14]:
# fill in CRSElapsedTime with mean
X_train_full = X_train.copy()
X_test_full = X_test.copy()

X_train_full['CRSElapsedTime'] = X_train['CRSElapsedTime'].fillna(X_train['CRSElapsedTime'].mean())
X_test_full['CRSElapsedTime'] = X_test['CRSElapsedTime'].fillna(X_test['CRSElapsedTime'].mean())

In [15]:
# Fill out CancellationCode with 'F' (assuming the flight wasn't cancelled for all na)
X_train_full['CancellationCode'] = X_train['CancellationCode'].fillna('F')
X_test_full['CancellationCode'] = X_test['CancellationCode'].fillna('F')

## Encode categorical data

In [16]:
# Label encoding

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Initialize into new df's
Xle_train = X_train_full.copy()
Xle_test = X_test_full.copy()

# Fill out new df with transform
for col in X_cat_better.columns:
    Xle_train[col] = le.fit_transform(X_train_full[col])
    Xle_test[col] = le.transform(X_test_full[col])   

# Modelling

## Try Logistic Regression Mdl

In [17]:
# Build basic LogReg model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

mdlLrFly = LogisticRegression(random_state=1)
mdlLrFly.fit(Xle_train, y_train)
y_test_pred = mdlLrFly.predict(Xle_test)
print(classification_report(y_test, y_test_pred))
print("ROC_AUC_SCORE:")
print(roc_auc_score(y_test, y_test_pred))

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

         0.0       0.80      1.00      0.89     90105
         1.0       0.00      0.00      0.00     22771
         2.0       0.95      0.97      0.96      1128

    accuracy                           0.80    114004
   macro avg       0.58      0.66      0.62    114004
weighted avg       0.64      0.80      0.71    114004

ROC_AUC_SCORE:


ValueError: multiclass format is not supported

In [18]:
y_test_pred.mean()

0.020262446931686606

#### The model will predict that almost no flight is delayed 15 min and still gets 80% right

In [19]:
# Build LogReg model w/ balanced class weight
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

mdlLrFly = LogisticRegression(class_weight='balanced', random_state=1)
mdlLrFly.fit(Xle_train, y_train)
y_test_pred = mdlLrFly.predict(Xle_test)

print(classification_report(y_test, y_test_pred))
print("ROC_AUC_SCORE:")
print(roc_auc_score(y_test, y_test_pred))

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

         0.0       0.81      0.96      0.88     90105
         1.0       0.37      0.10      0.15     22771
         2.0       0.95      1.00      0.98      1128

    accuracy                           0.79    114004
   macro avg       0.71      0.69      0.67    114004
weighted avg       0.72      0.79      0.73    114004

ROC_AUC_SCORE:


ValueError: multiclass format is not supported

In [20]:
y_test_pred.mean()

0.07207641837128521

#### This model doesn't perform well, but at least it is doing actual predicting

In [22]:
# Build LogReg model w/ increased number of iterations
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

mdlLrFly = LogisticRegression(class_weight='balanced', random_state=1, max_iter=1000)
mdlLrFly.fit(Xle_train, y_train)
y_test_pred = mdlLrFly.predict(Xle_test)

print(classification_report(y_test, y_test_pred))
print("ROC_AUC_SCORE:")
print(roc_auc_score(y_test, y_test_pred))

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

         0.0       0.86      0.58      0.69     91233
         1.0       0.27      0.61      0.37     22771

    accuracy                           0.59    114004
   macro avg       0.56      0.59      0.53    114004
weighted avg       0.74      0.59      0.63    114004

ROC_AUC_SCORE:
0.5944625708726723


#### Increasing the nubmer of iterations did not improve performance

## Try with a Random Forest Mdl

In [26]:
from sklearn.ensemble import RandomForestClassifier
   
mdlRfsFly = RandomForestClassifier(random_state=1, class_weight='balanced')
mdlRfsFly.fit(Xle_train, y_train)
y_test_pred = mdlRfsFly.predict(Xle_test)
class_rept = classification_report(y_test, y_test_pred)
      
print(class_rept)
print("ROC_AUC_SCORE:")
print(roc_auc_score(y_test, y_test_pred))

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

         0.0       0.82      0.95      0.88     91233
         1.0       0.44      0.16      0.23     22771

    accuracy                           0.79    114004
   macro avg       0.63      0.55      0.56    114004
weighted avg       0.74      0.79      0.75    114004

ROC_AUC_SCORE:
0.5534304593404727


#### 79% accuracy, but it still does very poorly in getting 1's correct, and has a lower AUC score than logistic regression

In [29]:
mdlRfsFly.n_features_

16

In [32]:
# Improve Rfs by increasing max_features and n_estimators
from sklearn.ensemble import RandomForestClassifier
   
mdlRfsFly = RandomForestClassifier(random_state=1, class_weight='balanced', max_features=8, n_estimators=100)
mdlRfsFly.fit(Xle_train, y_train)
y_test_pred = mdlRfsFly.predict(Xle_test)
class_rept = classification_report(y_test, y_test_pred)
      
print(class_rept)
print("ROC_AUC_SCORE:")
print(roc_auc_score(y_test, y_test_pred))

              precision    recall  f1-score   support

         0.0       0.82      0.96      0.88     91233
         1.0       0.50      0.17      0.25     22771

    accuracy                           0.80    114004
   macro avg       0.66      0.56      0.57    114004
weighted avg       0.76      0.80      0.76    114004

ROC_AUC_SCORE:
0.5621963957570029


#### That didn't improve much

## Try Grad Boosted Tree

In [34]:
from sklearn.ensemble import GradientBoostingClassifier

mdlGbcFly = GradientBoostingClassifier(random_state=1)
mdlGbcFly.fit(Xle_train, y_train)
y_test_pred = mdlGbcFly.predict(Xle_test)
print(classification_report(y_test, y_test_pred))
print("ROC_AUC_SCORE:")
print(roc_auc_score(y_test, y_test_pred))

              precision    recall  f1-score   support

         0.0       0.80      1.00      0.89     91233
         1.0       0.60      0.02      0.04     22771

    accuracy                           0.80    114004
   macro avg       0.70      0.51      0.47    114004
weighted avg       0.76      0.80      0.72    114004

ROC_AUC_SCORE:
0.5088573239247914


#### This model does not have a way of balancing class weights, so I won't further explore it.

## Try with Naive Baise

#### I don't think our features could be assumed to be independent however

### Gaussian

In [34]:
from sklearn.naive_bayes import GaussianNB

mdlNbgFly = GaussianNB()
mdlNbgFly.fit(Xle_train,y_train)
y_test_pred = mdlNbgFly.predict(Xle_test)
print('Results (Naive Bayes - Gaussian):\n')
print('Classification Report:')
print(classification_report(y_test, y_test_pred))
print('AUC score:')
print(roc_auc_score(y_test, y_test_pred))

Results (Naive Bayes - Gaussian):

Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.54      0.67     91233
         1.0       0.26      0.64      0.37     22771

    accuracy                           0.56    114004
   macro avg       0.56      0.59      0.52    114004
weighted avg       0.74      0.56      0.61    114004

AUC score:
0.5931178484390229


### Bernoulli

In [33]:
from sklearn.naive_bayes import BernoulliNB

# Define
mdlNbbFly = BernoulliNB()
# Fit
mdlNbbFly.fit(Xle_train,y_train)
# Predict
y_test_pred = mdlNbbFly.predict(Xle_test)
# Eval.
print('Results (Naive Bayes - Bernoulli):\n')
print('Classification Report:')
print(classification_report(y_test, y_test_pred))
print('AUC score:')
print(roc_auc_score(y_test, y_test_pred))

Results (Naive Bayes - Bernoulli):

Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      1.00      0.89     91233
         1.0       1.00      0.00      0.00     22771

    accuracy                           0.80    114004
   macro avg       0.90      0.50      0.44    114004
weighted avg       0.84      0.80      0.71    114004

AUC score:
0.5000439155065654


### Multinomial

In [36]:
from sklearn.naive_bayes import MultinomialNB

mdlNbmFly = MultinomialNB()
mdlNbmFly.fit(Xle_train,y_train)
y_test_pred = mdlNbmFly.predict(Xle_test)

print('Results (Naive Bayes - Multinomial):\n')
print('Classification Report:')
print(classification_report(y_test, y_test_pred))
print('AUC score:')
print(roc_auc_score(y_test, y_test_pred))

ValueError: Input X must be non-negative

## Try with a Neural Network

In [ ]:
%%time
from sklearn.neural_network import MLPClassifier

mdlNncFly = MLPClassifier(random_state=1)
mdlNncFly.fit(Xle_train, y_train)
y_test_pred = mdlNncFly.predict(Xle_test)

print('Classification Report (Neural Network):')
print(classification_report(y_test, y_test_pred))

## Try with a SVC model

In [ ]:
%%time
from sklearn.svm import SVC

mdlSvmFly = SVC(random_state=1, gamma='scale')
mdlSvmFly.fit(Xle_train, y_train)
y_test_pred = mdlSvmFly.predict(Xle_test)

print('Classification Report (Support Vector Classifier):')
print(classification_report(y_test, y_test_pred))

### Neural Networks and SVC took too much time to run

#### In summary, the Logistic Regression and Niave Baise Gaussian models performed best in terms of AUC scores both with good recall scores compared to other models. One can achieve the best accuracy by simply predicting that no flight will be delayed more than 15 min since our that outcome comprises only about 20% of our data. This would not be meaninngful prediction, however. 